In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2023-12-18 02:07:18.219070: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 02:07:18.249704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 02:07:18.249734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 02:07:18.250663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 02:07:18.256186: I tensorflow/core/platform/cpu_feature_guar

2.15.0


#### Get example MNIST data

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### Make simple model

In [13]:
from keras import Sequential
from keras.layers import Dense, Dropout
def create_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,)),
        Dropout(0.2),
        Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Save Checkpoints During Training
- `tf.keras.callbacks.ModelCheckpoint`
- latest checkpoint: `tf.train.latest_checkpoint`

In [14]:
checkpoint_path = "../model_checkpoints/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Make checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# Train with ckpt callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10


27/32 [========================>.....] - ETA: 0s - loss: 1.1802 - sparse_categorical_accuracy: 0.6725
Epoch 1: saving model to ../model_checkpoints/training_1/cp.ckpt
32/32 [==============================] - 1s 18ms/step - loss: 1.1236 - sparse_categorical_accuracy: 0.6840 - val_loss: 0.6773 - val_sparse_categorical_accuracy: 0.7900
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4081 - sparse_categorical_accuracy: 0.8901
Epoch 2: saving model to ../model_checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.4072 - sparse_categorical_accuracy: 0.8910 - val_loss: 0.5094 - val_sparse_categorical_accuracy: 0.8370
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2725 - sparse_categorical_accuracy: 0.9354
Epoch 3: saving model to ../model_checkpoints/training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.2753 - sparse_categorical_accuracy: 0.9350 - val_loss: 0.4524 - val_sparse_ca

In [15]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

#### Load Weights
- `model.load_weights`

In [16]:
# As long as two models have the same architecture, you can load the weights from one to the other
# Create basic model instance
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 2.3632 - sparse_categorical_accuracy: 0.1050 - 154ms/epoch - 5ms/step
Untrained model, accuracy: 10.50%


In [17]:
# Load weights
model.load_weights(checkpoint_path)

# Evaluate model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4021 - sparse_categorical_accuracy: 0.8650 - 142ms/epoch - 4ms/step
Untrained model, accuracy: 86.50%


In [19]:
# Extra callback options
# Save every 5 epochs
checkpoint_path = '../model_checkpoints/training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size=32

# Get batches per epoch
import math
n_batches = len(train_images)/batch_size
n_batches = math.ceil(n_batches) # Probably could've done this with // or something

# Create callback that saves model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq = 5*n_batches
)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0)) # Save weights with checkpoint path format

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)



Epoch 5: saving model to ../model_checkpoints/training_2/cp-0005.ckpt

Epoch 10: saving model to ../model_checkpoints/training_2/cp-0010.ckpt

Epoch 15: saving model to ../model_checkpoints/training_2/cp-0015.ckpt

Epoch 20: saving model to ../model_checkpoints/training_2/cp-0020.ckpt

Epoch 25: saving model to ../model_checkpoints/training_2/cp-0025.ckpt

Epoch 30: saving model to ../model_checkpoints/training_2/cp-0030.ckpt

Epoch 35: saving model to ../model_checkpoints/training_2/cp-0035.ckpt

Epoch 40: saving model to ../model_checkpoints/training_2/cp-0040.ckpt

Epoch 45: saving model to ../model_checkpoints/training_2/cp-0045.ckpt

Epoch 50: saving model to ../model_checkpoints/training_2/cp-0050.ckpt


In [20]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'checkpoint',
 'cp-0010.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [21]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'../model_checkpoints/training_2/cp-0050.ckpt'

In [22]:
# Reset and redo
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4788 - sparse_categorical_accuracy: 0.8750 - 160ms/epoch - 5ms/step
Restored model, accuracy: 87.50%


#### Manual Save Weights
- `save_weights` and `load_weights`

In [23]:
model.save_weights('../model_checkpoints/test_checkpoint')
model = create_model()
model.load_weights('../model_checkpoints/test_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4788 - sparse_categorical_accuracy: 0.8750 - 158ms/epoch - 5ms/step
Restored model, accuracy: 87.50%


#### Save Full Model
- New `.keras` format
- Legacy `SavedModel` and `HDF5` formats
- Custom objects like subclassed models or layers get saved differently

In [26]:
# .keras format
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('../saved_models/keras_save.keras')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.2399 - sparse_categorical_accuracy: 0.6480
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4518 - sparse_categorical_accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2907 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2289 - sparse_categorical_accuracy: 0.9430
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1692 - sparse_categorical_accuracy: 0.9590


In [28]:
# Refresh
new_model = tf.keras.models.load_model('../saved_models/keras_save.keras')
new_model.summary()
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
32/32 - 0s - loss: 0.4229 - sparse_categorical_accuracy: 0.8730 - 167ms/epoch - 5ms/step
Restored model, accuracy: 87.30%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


In [29]:
# SavedModel format
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
model.save('../saved_models/saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1639 - sparse_categorical_accuracy: 0.6730
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4146 - sparse_categorical_accuracy: 0.8890
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2771 - sparse_categorical_accuracy: 0.9390
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2066 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1541 - sparse_categorical_accuracy: 0.9660
INFO:tensorflow:Assets written to: ../saved_models/saved_model/my_model/assets


INFO:tensorflow:Assets written to: ../saved_models/saved_model/my_model/assets


In [31]:
!ls ../saved_models/saved_model
!ls ../saved_models/saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [33]:
# Reload
new_model = tf.keras.models.load_model('../saved_models/saved_model/my_model')

# Check its architecture
new_model.summary()
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
32/32 - 0s - loss: 0.4308 - sparse_categorical_accuracy: 0.8600 - 178ms/epoch - 6ms/step
Restored model, accuracy: 86.00%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


In [34]:
# HDF5 format
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('../saved_models/hdf5_test_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1458 - sparse_categorical_accuracy: 0.6900
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4132 - sparse_categorical_accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2790 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1980 - sparse_categorical_accuracy: 0.9580
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1551 - sparse_categorical_accuracy: 0.9710


/home/kmirijan/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
new_model = tf.keras.models.load_model('../saved_models/hdf5_test_model.h5')

# Show the model architecture
new_model.summary()
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 512)               401920    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
32/32 - 0s - loss: 0.4070 - sparse_categorical_accuracy: 0.8640 - 164ms/epoch - 5ms/step
Restored model, accuracy: 86.40%


#### Custom Objects
- Custom objects can be saved as `SavedModels` without any special care, but they are harder to debug
- Check the tutorial for saving as `.keras` and `HDF5` since they are "better"